In [1]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'AWA2'
path = 'E:/Sushree/Dataset/data/xlsa17/data/'

res101 = scipy.io.loadmat(path + dataset + '/res101.mat')
att_splits = scipy.io.loadmat(path + dataset + '/att_splits.mat')

In [3]:
signature = att_splits['att']
#signature = att_splits['original_att']
#signature = signature/100
print(signature.shape) #(85, 50)

attribute = signature.transpose()
attribute[attribute<0] = 0
print(attribute, attribute.shape)#(50, 85)

(85, 50)
[[0.         0.         0.         ... 0.00882092 0.03640974 0.03145501]
 [0.12045618 0.00426584 0.         ... 0.17996306 0.0618086  0.03495531]
 [0.26584459 0.20652363 0.         ... 0.05026822 0.04274552 0.04915256]
 ...
 [0.22516498 0.15266022 0.         ... 0.12733492 0.10009694 0.01771   ]
 [0.19613947 0.1966714  0.         ... 0.01787277 0.06698743 0.25883601]
 [0.03819588 0.08046548 0.10363715 ... 0.01479997 0.05250999 0.14194515]] (50, 85)


In [4]:
import gensim.downloader as api
import pandas as pd
print('Load pretrain w2v model')

import transformers
from transformers import BertTokenizer, TFBertModel
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = TFBertModel.from_pretrained("bert-base-uncased")

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = TFBertModel.from_pretrained("bert-large-uncased")

dim_w2v = 1024

#%%
replace_word = [('newworld','new world'),('oldworld','old world'),('nestspot','nest spot'),('toughskin','tough skin'),
                ('longleg','long leg'), ('longneck', 'long neck'), ('chewteeth','chew teeth'),('meatteeth','meat teeth'),('strainteeth','strain teeth'),
                ('quadrapedal','quadrupedal')]  # for AWA2


#For AWA2
path = 'E:/Sushree/Dataset/Animals_with_Attributes2/attribute/predicates.txt'
df=pd.read_csv(path,sep='\t',header = None, names = ['idx','des'])
des = df['des'].values

#%% replace out of dictionary (OOD) words
for pair in replace_word:
    for idx,s in enumerate(des):
        des[idx] = s.replace(pair[0],pair[1])
print('Done replacing OOD words')

df['new_des'] = des
df.to_csv('E:/Sushree/Dataset/Animals_with_Attributes2/attribute/new_des.csv')
print('Done preprocessing attribute des')

import pickle

counter = 0

w2v_att = np.zeros((signature.shape[0], dim_w2v))
for s in des:
    print(s)
    w2v = np.zeros(dim_w2v)
    encoded_input = tokenizer(s, return_tensors='tf')
    length = encoded_input.input_ids.shape[1]
    #print(length)
    for i in range(length-2):
        w2v = w2v + model(encoded_input).last_hidden_state[:, i+1, :]
        #print(model(encoded_input).last_hidden_state[:, i+1, :][:,1:2])
        #print(w2v[:, 1:2])
        #print(w2v.shape)

    w2v = w2v / (length - 2)
    #print(w2v[:, 1:2])
    w2v_att[counter] = w2v
    counter = counter + 1

print('counter ',counter)

print(w2v_att, w2v_att.shape)

Load pretrain w2v model


C:\Users\Admin\anaconda3\envs\en3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassifica

Done replacing OOD words
Done preprocessing attribute des
black
white
blue
brown
gray
orange
red
yellow
patches
spots
stripes
furry
hairless
tough skin
big
small
bulbous
lean
flippers
hands
hooves
pads
paws
long leg
long neck
tail
chew teeth
meat teeth
buckteeth
strain teeth
horns
claws
tusks
smelly
flys
hops
swims
tunnels
walks
fast
slow
strong
weak
muscle
bipedal
quadrupedal
active
inactive
nocturnal
hibernate
agility
fish
meat
plankton
vegetation
insects
forager
grazer
hunter
scavenger
skimmer
stalker
new world
old world
arctic
coastal
desert
bush
plains
forest
fields
jungle
mountains
ocean
ground
water
tree
cave
fierce
timid
smart
group
solitary
nest spot
domestic
counter  85
[[-0.09144888 -0.13553846 -0.12541848 ... -0.02989655 -0.38680694
   0.24406666]
 [-0.29638147  0.14530022  0.20919298 ... -0.13767773 -0.49952391
  -0.02497395]
 [-0.17159468  0.01720696 -0.01027012 ...  0.11106262  0.08405223
   0.08059736]
 ...
 [-0.36489192  0.06430094 -0.17526677 ...  0.12331312 -0.369820

In [5]:
attribute_new = np.einsum('ij,jl->ijl', attribute, w2v_att)

attribute_new = np.reshape(attribute_new, [attribute_new.shape[0], attribute_new.shape[1]* attribute_new.shape[2]])
print(attribute_new, attribute_new.shape)

#attribute_new[attribute_new<0] = 0
#print(attribute_new, attribute_new.shape)

[[ 0.          0.          0.         ... -0.00612236 -0.01822051
   0.01286502]
 [-0.01101558 -0.01632644 -0.01510743 ... -0.00680365 -0.02024807
   0.01429663]
 [-0.02431119 -0.03603217 -0.03334183 ... -0.00956699 -0.02847192
   0.02010326]
 ...
 [-0.02059108 -0.03051851 -0.02823985 ... -0.00344705 -0.01025863
   0.00724334]
 [-0.01793673 -0.02658444 -0.02459951 ... -0.05037949 -0.14993233
   0.10586323]
 [-0.00349297 -0.00517701 -0.00479047 ... -0.02762801 -0.08222259
   0.05805518]] (50, 87040)


In [6]:
# total number of instances or images = 37322: ranges from 0 to 37321

trainval_loc = np.squeeze(att_splits['trainval_loc']-1) # -1: to consider the overflow problem
print(np.unique(trainval_loc), np.max(np.unique(trainval_loc))) # smallest location: 2, largest location 37321

test_seen_loc = np.squeeze(att_splits['test_seen_loc']-1)
print(np.unique(test_seen_loc), np.max(np.unique(test_seen_loc))) # smallest location: 0, largest location 37319

test_unseen_loc = np.squeeze(att_splits['test_unseen_loc']-1)
print(np.unique(test_unseen_loc), np.max(np.unique(test_unseen_loc))) # smallest location: 1046, largest location 35290



[    2     4     5 ... 37318 37320 37321] 37321
[    0     1     3 ... 37306 37307 37319] 37319
[ 1046  1047  1048 ... 35288 35289 35290] 35290


In [7]:

labels = res101['labels']# direct class labels
print('labels', labels, labels.shape)# 37322 x 1

print('unique_labels', np.unique(labels), np.unique(labels).shape)# class labels range from 1 to 50, 50 classes

# get the labels for trainval, test seen and test unseen sets

labels_trainval = labels[trainval_loc]
print('labels_trainval', labels_trainval, labels_trainval.shape)

unique_labels_trainval = np.unique(labels_trainval) # labels min:1 max:49
print('unique_labels_trainval', unique_labels_trainval, unique_labels_trainval.shape)# 40 classes


labels_test_seen = labels[test_seen_loc]
print('labels_test_seen', labels_test_seen, labels_test_seen.shape)

unique_labels_test_seen = np.unique(labels_test_seen) # labels min:1 max:49
print('unique_labels_test_seen', unique_labels_test_seen, unique_labels_test_seen.shape)# 40 classes


labels_test_unseen = labels[test_unseen_loc]
print('labels_test_unseen', labels_test_unseen, labels_test_unseen.shape)

unique_labels_test_unseen = np.unique(labels_test_unseen) # labels min:7 max:50
print('unique_labels_test_unseen', unique_labels_test_unseen, unique_labels_test_unseen.shape)# 10 classes


if len(labels) == len(labels_trainval) + len(labels_test_seen) + len(labels_test_unseen):
    print('correct number of instances for training, test seen and test unseen categories')
    
print("Number of overlapping classes between trainval and test seen:",len(set(unique_labels_trainval).intersection(set(unique_labels_test_seen))))

print("Number of overlapping classes between trainval and test unseen:",len(set(unique_labels_trainval).intersection(set(unique_labels_test_unseen))))


labels [[ 1]
 [ 1]
 [ 1]
 ...
 [38]
 [38]
 [38]] (37322, 1)
unique_labels [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] (50,)
labels_trainval [[43]
 [22]
 [43]
 ...
 [40]
 [19]
 [46]] (23527, 1)
unique_labels_trainval [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)
labels_test_seen [[22]
 [49]
 [14]
 ...
 [25]
 [15]
 [27]] (5882, 1)
unique_labels_test_seen [ 1  2  3  4  5  6  8 10 11 12 13 14 15 16 17 18 19 20 21 22 25 26 27 28
 29 32 33 35 36 37 38 39 40 42 43 44 45 46 48 49] (40,)
labels_test_unseen [[30]
 [30]
 [30]
 ...
 [47]
 [47]
 [47]] (7913, 1)
unique_labels_test_unseen [ 7  9 23 24 30 31 34 41 47 50] (10,)
correct number of instances for training, test seen and test unseen categories
Number of overlapping classes between trainval and test seen: 40
Number of overlapping classes between trainv

In [8]:
X_features = res101['features']

# locations are already subtracted by 1, so they range from 0 to 37321
trainval_vec = X_features[:, trainval_loc].transpose()
test_seen_vec = X_features[:, test_seen_loc].transpose()
test_unseen_vec = X_features[:, test_unseen_loc].transpose()

print("Features for trainval:", trainval_vec.shape) #(23527, 2048)
print("Features for test seen:", test_seen_vec.shape)# (5882, 2048)
print("Features for test unseen:", test_unseen_vec.shape) #(7913, 2048)

Features for trainval: (23527, 2048)
Features for test seen: (5882, 2048)
Features for test unseen: (7913, 2048)


In [9]:
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import *
#from keras.optimizers import SGD, Adam, Adagrad
#import keras.backend as K

#inputt = Input(shape = trainval_vec.shape[1])
#hidden = Dense(2048, name="layer1", activation='linear')(inputt)
#output = Dense(1024, name="layer3", activation='linear')(hidden)

#model00 = Model(inputs = inputt, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
#opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

#model00.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

#for layer in model00.layers:
 #       layer.trainable = False
        
#model00.summary()

In [10]:
#trainval_vec = model00.predict(trainval_vec)
#print(trainval_vec, trainval_vec.shape)

#test_seen_vec = model00.predict(test_seen_vec)
#print(test_seen_vec, test_seen_vec.shape)

#test_unseen_vec = model00.predict(test_unseen_vec)
#print(test_unseen_vec, test_unseen_vec.shape)

In [11]:


# attribute is defined for all 50 classes, so we cant use locations directly, instead we have to use labels 
# that range from 1 to 50, so we have to subtract 1

train_attributes = np.zeros((len(trainval_loc), attribute_new.shape[1]))
for i in range(len(trainval_loc)):
    train_attributes[i] = attribute_new[int(labels_trainval[i])-1]

print(train_attributes, train_attributes.shape)# (23527, 85)

test_seen_attributes = np.zeros((len(test_seen_loc), attribute_new.shape[1]))
for i in range(len(test_seen_loc)):
    test_seen_attributes[i] = attribute_new[int(labels_test_seen[i])-1]

print(test_seen_attributes, test_seen_attributes.shape)# (5882, 85)

#test_unseen_attributes = np.zeros((len(test_unseen_loc), attribute_new.shape[1]))
#for i in range(len(test_unseen_loc)):
#    test_unseen_attributes[i] = attribute_new[int(labels_test_unseen[i])-1]

#print(test_unseen_attributes, test_unseen_attributes.shape)# (7913, 85)


[[-0.00052664 -0.00078054 -0.00072226 ... -0.00223581 -0.00665391
   0.00469815]
 [ 0.          0.          0.         ... -0.00277371 -0.00825473
   0.00582844]
 [-0.00052664 -0.00078054 -0.00072226 ... -0.00223581 -0.00665391
   0.00469815]
 ...
 [ 0.          0.          0.         ... -0.0064743  -0.01926791
   0.01360456]
 [-0.00076979 -0.00114092 -0.00105574 ... -0.00450891 -0.01341877
   0.00947463]
 [-0.00354577 -0.00525526 -0.00486288 ... -0.05893629 -0.17539784
   0.12384375]] (23527, 87040)
[[ 0.          0.          0.         ... -0.00277371 -0.00825473
   0.00582844]
 [-0.01793673 -0.02658444 -0.02459951 ... -0.05037949 -0.14993233
   0.10586323]
 [-0.00176324 -0.00261333 -0.00241821 ...  0.          0.
   0.        ]
 ...
 [-0.01226115 -0.01817253 -0.01681568 ... -0.02011478 -0.05986277
   0.04226751]
 [-0.01070116 -0.01586043 -0.01467621 ... -0.00348217 -0.01036316
   0.00731715]
 [-0.00328189 -0.00486417 -0.00450099 ... -0.00976332 -0.02905622
   0.02051582]] (5882, 87

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad
import keras.backend as K

attribute_shape_new = attribute_new.shape[1]
print(attribute_shape_new)

# define model for attribute transformation

inputt = Input(shape = attribute_shape_new)
hidden = Dense(2048, name="layer1", activation='linear')(inputt)
#norm_layer = Lambda(lambda x: K.l2_normalize(x,axis=1))
#batchnorm = BatchNormalization()(hidden)
#batchnorm = norm_layer(hidden)
#hidden2 = Dense(1024, name="layer2", activation='linear')(hidden)
output = Dense(512, name="layer3", activation='linear')(hidden)

model0 = Model(inputs = inputt, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model0.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

for layer in model0.layers:
        layer.trainable = False
        
model0.summary()

87040
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 87040)]           0         
                                                                 
 layer1 (Dense)              (None, 2048)              178259968 
                                                                 
 layer3 (Dense)              (None, 512)               1049088   
                                                                 
Total params: 179,309,056
Trainable params: 0
Non-trainable params: 179,309,056
_________________________________________________________________


In [13]:
train_attributes_2 = model0.predict(train_attributes)
print(train_attributes_2, train_attributes_2.shape)

test_seen_attributes_2 = model0.predict(test_seen_attributes)
print(test_seen_attributes_2, test_seen_attributes_2.shape)

#test_unseen_attributes_2 = model0.predict(test_unseen_attributes)
#print(test_unseen_attributes_2, test_unseen_attributes_2.shape)

736/736 [==============================] - 4s 5ms/step
[[-0.01060587  0.16492726  0.13291374 ... -0.12251601  0.0395489
   0.02072064]
 [-0.015154    0.03513538  0.05185136 ... -0.09346648 -0.01137062
   0.03519592]
 [-0.01060587  0.16492726  0.13291374 ... -0.12251601  0.0395489
   0.02072064]
 ...
 [-0.0617771   0.01114547  0.01065801 ... -0.09507618 -0.02953161
   0.17195356]
 [-0.01827106  0.04678195  0.07453287 ... -0.11931957  0.03731982
   0.17031279]
 [-0.0517026   0.10743117  0.01382869 ... -0.02287627 -0.03851479
   0.05824069]] (23527, 512)
184/184 [==============================] - 1s 5ms/step
[[-0.015154    0.03513538  0.05185136 ... -0.09346648 -0.01137062
   0.03519592]
 [-0.04835539  0.08162045  0.05941594 ... -0.06856067 -0.03110444
   0.20423575]
 [-0.01049893  0.03613483  0.06606321 ... -0.12659201 -0.00203105
   0.10977834]
 ...
 [ 0.05245933  0.05708604  0.04050728 ... -0.10841252  0.11985397
   0.07459512]
 [-0.0106774   0.14739555  0.09137649 ... -0.08296686  0.0

In [14]:
# as labels range from 1 to 50, we have subtract 1
attribute_2 = model0.predict(attribute_new)
print(attribute_2, attribute_2.shape)

signature_2 = attribute_2.transpose()
print(signature_2, signature_2.shape)#(50, 300)

trainval_sig = signature_2[:, (unique_labels_trainval)-1]
test_seen_sig = signature_2[:, (unique_labels_test_seen)-1]
test_unseen_sig = signature_2[:, (unique_labels_test_unseen)-1]

print("Signature for trainval:", trainval_sig.shape)
print("Signature for test seen:", test_seen_sig.shape)
print("Signature for test unseen:", test_unseen_sig.shape)

2/2 [==============================] - 0s 3ms/step
[[-0.04613888 -0.06162656  0.06225181 ... -0.08769844 -0.00829026
   0.16642572]
 [-0.00463901  0.21757042  0.07664593 ... -0.01963473 -0.07946007
  -0.01018086]
 [-0.00319296  0.08191028  0.11915039 ... -0.03012314  0.07096008
   0.1303697 ]
 ...
 [ 0.00663882  0.02915052  0.01089406 ... -0.06127711 -0.02300003
   0.06792159]
 [-0.04835549  0.08162059  0.05941591 ... -0.06856066 -0.03110447
   0.20423594]
 [ 0.03697511 -0.06499453  0.06677884 ... -0.07991131  0.08740322
   0.19930871]] (50, 512)
[[-0.04613888 -0.00463901 -0.00319296 ...  0.00663882 -0.04835549
   0.03697511]
 [-0.06162656  0.21757042  0.08191028 ...  0.02915052  0.08162059
  -0.06499453]
 [ 0.06225181  0.07664593  0.11915039 ...  0.01089406  0.05941591
   0.06677884]
 ...
 [-0.08769844 -0.01963473 -0.03012314 ... -0.06127711 -0.06856066
  -0.07991131]
 [-0.00829026 -0.07946007  0.07096008 ... -0.02300003 -0.03110447
   0.08740322]
 [ 0.16642572 -0.01018086  0.1303697 

In [15]:
# by doing this modification, we are changing the range of trainval and test seen labels from 0 to 39 
# and test unseen labels from 0 to 9

k = 0
new_labels_trainval = np.zeros((len(labels_trainval), 1), dtype = 'int')
for labels in unique_labels_trainval:
    new_labels_trainval[labels_trainval == labels] = k
    k = k+1
    
print(new_labels_trainval, new_labels_trainval.shape)#(23527, 1)

l = 0
new_labels_test_seen = np.zeros((len(labels_test_seen), 1), dtype = 'int')
for labels in unique_labels_test_seen:
    new_labels_test_seen[labels_test_seen == labels] = l
    l = l+1
    
print(new_labels_test_seen, new_labels_test_seen.shape)# (5882, 1)

m = 0
new_labels_test_unseen = np.zeros((len(labels_test_unseen), 1), dtype = 'int')
for labels in unique_labels_test_unseen:
    new_labels_test_unseen[labels_test_unseen == labels] = m
    m = m+1  

print(new_labels_test_unseen, new_labels_test_unseen.shape) #  (7913, 1)  


print(np.unique(new_labels_trainval), np.unique(new_labels_trainval).shape)

print(np.unique(new_labels_test_seen), np.unique(new_labels_test_seen).shape)

print(np.unique(new_labels_test_unseen), np.unique(new_labels_test_unseen).shape)

[[34]
 [19]
 [34]
 ...
 [32]
 [16]
 [37]] (23527, 1)
[[19]
 [39]
 [11]
 ...
 [20]
 [12]
 [22]] (5882, 1)
[[4]
 [4]
 [4]
 ...
 [8]
 [8]
 [8]] (7913, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] (40,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39] (40,)
[0 1 2 3 4 5 6 7 8 9] (10,)


In [16]:
#params for trainval and test set
m_trainval = new_labels_trainval.shape[0]# number of instances in training set: 23527
print(m_trainval)

z_trainval = len(unique_labels_trainval)# number of classes in training set: 40
print(z_trainval)


n_test_seen = new_labels_test_seen.shape[0]# 5882
print(n_test_seen)

z1_test_seen = len(unique_labels_test_seen)# 40
print(z1_test_seen)


n_test_unseen = new_labels_test_unseen.shape[0]# 7913
print(n_test_unseen)

z1_test_unseen = len(unique_labels_test_unseen)# 10
print(z1_test_unseen)

23527
40
5882
40
7913
10


In [17]:
from tensorflow.keras.utils import to_categorical
gt_trainval = to_categorical(new_labels_trainval, z_trainval)

print(gt_trainval, gt_trainval.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]] (23527, 40)


In [18]:
input1_shape = trainval_vec.shape[1]
print(input1_shape)

attribute_shape = trainval_sig.shape[0]
print(attribute_shape)

output_shape = z_trainval
print(output_shape)


2048
512
40


In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.optimizers import SGD, Adam, Adagrad

# define model2 for attribute to class label mapping

input2 = Input(shape = (1, 1, attribute_shape))
flat = Flatten()(input2)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model2 = Model(inputs = input2, outputs = output)

#sgd = SGD(learning_rate = 1e-2, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model2.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 1, 512)]       0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 output (Dense)              (None, 40)                20520     
                                                                 
Total params: 20,520
Trainable params: 20,520
Non-trainable params: 0
_________________________________________________________________


In [20]:
# define model1 for resnet feature to class label mapping

input1 = Input(shape = (1, 1, input1_shape))
#inter_pre = Dense(512, name="intermediate_previous", activation='relu')(input1)
inter = Conv1D(attribute_shape, kernel_size = 1, name = "intermediate", activation = 'linear')(input1)
flat = Flatten()(inter)
output = Dense(output_shape, name="output", activation='softmax')(flat)

model1 = Model(inputs = input1, outputs = output)

opt = Adam(learning_rate = 1e-2, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0001)

model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 intermediate (Conv1D)       (None, 1, 1, 512)         1049088   
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 output (Dense)              (None, 40)                20520     
                                                                 
Total params: 1,069,608
Trainable params: 1,069,608
Non-trainable params: 0
_________________________________________________________________


In [21]:
trainval_input1 = np.reshape(trainval_vec, [trainval_vec.shape[0], 1, 1, trainval_vec.shape[1]])
print(trainval_input1.shape)

trainval_input2 = np.reshape(train_attributes_2, [train_attributes_2.shape[0], 1, 1, train_attributes_2.shape[1]])
print(trainval_input2.shape)

trainval_output = gt_trainval
print(trainval_output.shape)

(23527, 1, 1, 2048)
(23527, 1, 1, 512)
(23527, 40)


In [22]:
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
batch_size = 16
from sklearn.model_selection import train_test_split    

In [23]:

X_train1, X_val1, y_train1, y_val1 = train_test_split(trainval_input1, trainval_output, test_size = 0.2, random_state = 42)

#train_gen1 = DataGenerator(X_train1, y_train1, batch_size)   
#val_gen1 = DataGenerator(X_val1, y_val1, batch_size)


X_train2, X_val2, y_train2, y_val2 = train_test_split(trainval_input2, trainval_output, test_size = 0.2, random_state = 42)

#train_gen2 = DataGenerator(X_train2, y_train2, batch_size)   
#val_gen2 = DataGenerator(X_val2, y_val2, batch_size)

In [24]:
iteration = 15
epochs1 = 100
epochs2 = 200

best_performance_micro = [0, 0, 0]
best_performance_macro = [0, 0, 0]

save_path = 'C:/Users/Admin/Sushree_Codes/Sush_3/Results/'
name = 'model1_conv_AWA2_Bert2attT_1024_lin_it15_100eph_adam_cce_16bch_1e-2lr_model2_adam_lr-2_200'

for i in range(iteration):
    X_train2_it = X_train2[(len(X_train2)//iteration)*i:(len(X_train2)//iteration)*(i+1)]
    X_val2_it = X_val2[(len(X_val2)//iteration)*i:(len(X_val2)//iteration)*(i+1)]
    y_train2_it = y_train2[(len(y_train2)//iteration)*i:(len(y_train2)//iteration)*(i+1)]
    y_val2_it = y_val2[(len(y_val2)//iteration)*i:(len(y_val2)//iteration)*(i+1)]
    
    train_gen2 = DataGenerator(X_train2_it, y_train2_it, batch_size)   
    val_gen2 = DataGenerator(X_val2_it, y_val2_it, batch_size)

    train_summary2 = model2.fit(train_gen2, epochs = epochs2, verbose = 0, callbacks = None, validation_data = val_gen2, 
                              shuffle = True, steps_per_epoch = len(train_gen2)//batch_size, 
                              validation_steps = len(val_gen2)//batch_size)

    print("iteration:", i)
    print('model 2 is trained:', 'training acc:', train_summary2.history['accuracy'][-1], ',',  
          'training loss:', train_summary2.history['loss'][-1], ',', 
          'validation acc:', train_summary2.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary2.history['val_loss'][-1])

    weights_list2 = model2.get_weights()
    #print(weights_list2)

    model1.layers[-1].set_weights(weights_list2)

    X_train1_it = X_train1[(len(X_train1)//iteration)*i:(len(X_train1)//iteration)*(i+1)]
    X_val1_it = X_val1[(len(X_val1)//iteration)*i:(len(X_val1)//iteration)*(i+1)]
    y_train1_it = y_train1[(len(y_train1)//iteration)*i:(len(y_train1)//iteration)*(i+1)]
    y_val1_it = y_val1[(len(y_val1)//iteration)*i:(len(y_val1)//iteration)*(i+1)]
    
    train_gen1 = DataGenerator(X_train1_it, y_train1_it, batch_size)   
    val_gen1 = DataGenerator(X_val1_it, y_val1_it, batch_size)
    
    for layer in model1.layers[2:]:
        layer.trainable = True

    train_summary1 = model1.fit(train_gen1, epochs = epochs1, verbose = 0, callbacks = None, validation_data = val_gen1, 
                              shuffle = True, steps_per_epoch = len(train_gen1)//batch_size, 
                              validation_steps = len(val_gen1)//batch_size)
    
    #print("iteration:", i)
    print('model 1 is trained:', 'training acc:', train_summary1.history['accuracy'][-1], ',',  
          'training loss:', train_summary1.history['loss'][-1], ',', 
          'validation acc:', train_summary1.history['val_accuracy'][-1], ',',
         'validation_loss:', train_summary1.history['val_loss'][-1])
    
    weights_list1 = Model(inputs = model1.input, outputs = model1.layers[-1].output).get_weights()
    
    #predictions
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, np.matmul(weights_list1[0], weights_list1[2])), test_seen_sig)
    #outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0]), test_seen_sig)
    
    outputs_seen = np.matmul(np.matmul(test_seen_vec, weights_list1[0][0]), test_seen_sig)
    preds_seen = np.array([np.argmax(output) for output in outputs_seen])
    
    cm_seen = confusion_matrix(new_labels_test_seen, preds_seen)
    #print(cm)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_seen_micro = cm_seen.astype('float') / cm_seen.sum(axis=1)[:, np.newaxis]
    #print(cm)
    avg_seen_micro = (sum(cm_seen_micro.diagonal())/len(unique_labels_test_seen))*100

    avg_seen_macro = (sum(cm_seen.diagonal())/len(new_labels_test_seen))*100
    
    #predictions
    #outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0]), test_unseen_sig)
    outputs_unseen = np.matmul(np.matmul(test_unseen_vec, weights_list1[0][0]), test_unseen_sig)
    
    preds_unseen = np.array([np.argmax(output) for output in outputs_unseen])
    
    cm_unseen = confusion_matrix(new_labels_test_unseen, preds_unseen)
    # Compute macro average (averaging performance metrics by first calculating the metric separately for each class and 
    # then averaging these class-specific metrics)
    cm_unseen_micro = cm_unseen.astype('float') / cm_unseen.sum(axis=1)[:, np.newaxis]
    avg_unseen_micro = (sum(cm_unseen_micro.diagonal())/len(unique_labels_test_unseen))*100

    avg_unseen_macro = (sum(cm_unseen.diagonal())/len(new_labels_test_unseen))*100
    
    harmonic_micro = (2*avg_seen_micro*avg_unseen_micro) / (avg_seen_micro + avg_unseen_micro)
    harmonic_macro = (2*avg_seen_macro*avg_unseen_macro) / (avg_seen_macro + avg_unseen_macro)
    
    print('micro average')
    print('seen accuracy:', avg_seen_micro, 'unseen accuracy:', avg_unseen_micro, 'harmonic mean:', harmonic_micro)
    
    print('macro average')
    print('seen accuracy:', avg_seen_macro, 'unseen accuracy:', avg_unseen_macro, 'harmonic mean:', harmonic_macro)
    
    if harmonic_micro > best_performance_micro[2]:
        best_performance_micro = [avg_seen_micro, avg_unseen_micro, harmonic_micro]
        model1.save_weights(save_path + 'bw_micro_' + name + '.h5', overwrite=True)
        
    if harmonic_macro > best_performance_macro[2]:
        best_performance_macro = [avg_seen_macro, avg_unseen_macro, harmonic_macro]
        model1.save_weights(save_path + 'bw_macro_' + name + '.h5', overwrite=True)
        
    print('best accuracy micro','seen accuracy:', best_performance_micro[0], 'unseen accuracy:', best_performance_micro[1], 'harmonic mean:', best_performance_micro[2])
    print('best accuracy macro', 'seen accuracy:', best_performance_macro[0], 'unseen accuracy:', best_performance_macro[1], 'harmonic mean:', best_performance_macro[2])
    
    print('-----------------------------------------------------------------------------------------------------------')
    weights_list3 = model1.get_weights()
    model2.set_weights(weights_list3[2:])

iteration: 0
model 2 is trained: training acc: 0.984375 , training loss: 0.39786297082901 , validation acc: 0.9375 , validation_loss: 0.44718706607818604
model 1 is trained: training acc: 0.9375 , training loss: 4.351791858673096 , validation acc: 0.8125 , validation_loss: 36.395423889160156
micro average
seen accuracy: 72.3143098317929 unseen accuracy: 47.21888032397265 harmonic mean: 57.13226154524093
macro average
seen accuracy: 77.3206392383543 unseen accuracy: 41.273853153039305 harmonic mean: 53.819037381445185
best accuracy micro seen accuracy: 72.3143098317929 unseen accuracy: 47.21888032397265 harmonic mean: 57.13226154524093
best accuracy macro seen accuracy: 77.3206392383543 unseen accuracy: 41.273853153039305 harmonic mean: 53.819037381445185
-----------------------------------------------------------------------------------------------------------
iteration: 1
model 2 is trained: training acc: 1.0 , training loss: 0.1548878252506256 , validation acc: 1.0 , validation_loss:

iteration: 10
model 2 is trained: training acc: 1.0 , training loss: 0.035067081451416016 , validation acc: 1.0 , validation_loss: 0.04605497047305107
model 1 is trained: training acc: 0.984375 , training loss: 0.6672477722167969 , validation acc: 0.875 , validation_loss: 56.8167724609375
micro average
seen accuracy: 84.76937541375807 unseen accuracy: 48.02523876923816 harmonic mean: 61.31377420103535
macro average
seen accuracy: 88.72832369942196 unseen accuracy: 43.6876026791356 harmonic mean: 58.54775717966477
best accuracy micro seen accuracy: 86.05785487242682 unseen accuracy: 54.232326656416696 harmonic mean: 66.53520076645418
best accuracy macro seen accuracy: 89.20435226113567 unseen accuracy: 55.023379249336536 harmonic mean: 68.06353887358328
-----------------------------------------------------------------------------------------------------------
iteration: 11
model 2 is trained: training acc: 1.0 , training loss: 0.04050927981734276 , validation acc: 1.0 , validation_loss:

In [25]:
gt_test_seen = to_categorical(new_labels_test_seen, z1_test_seen)

print(gt_test_seen, gt_test_seen.shape)

test_seen_vec = np.reshape(test_seen_vec, [test_seen_vec.shape[0], 1, 1, test_seen_vec.shape[1]])
res1 = model1.evaluate(test_seen_vec, gt_test_seen)

p1 = model1.predict(test_seen_vec, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p1).numpy())


test_seen_attributes_2 = np.reshape(test_seen_attributes_2, [test_seen_attributes_2.shape[0], 1, 1, test_seen_attributes_2.shape[1]])

res2 = model2.evaluate(test_seen_attributes_2, gt_test_seen)

p2 = model2.predict(test_seen_attributes_2, verbose = 0)

import tensorflow
cce = tensorflow.keras.losses.CategoricalCrossentropy()
print('cce = ', cce(gt_test_seen, p2).numpy())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (5882, 40)
184/184 [==============================] - 0s 2ms/step - loss: 80.2680 - accuracy: 0.9004
cce =  1.5862501
184/184 [==============================] - 0s 1ms/step - loss: 0.0503 - accuracy: 1.0000
cce =  0.050263677


In [28]:

accuracy_seen_updated = res1[1]*100
unseen_accuracy =55.02
h = (2*accuracy_seen_updated*unseen_accuracy) / (accuracy_seen_updated + unseen_accuracy)
print(h)


accuracy_seen_updated2 = ((res1[1]*100)+(res2[1]*100))/2
print(accuracy_seen_updated2)
h = (2*accuracy_seen_updated2*unseen_accuracy) / (accuracy_seen_updated2 + unseen_accuracy)
print(h)

68.30203406651643
95.01869976520538
69.68773882022109


In [27]:
from sklearn.metrics import precision_recall_fscore_support

pp1 = np.array([np.argmax(output) for output in p1])
pp2 = np.array([np.argmax(output) for output in p2])

seen_macro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'macro')
seen_macro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'macro')
print('precision_seen_macro', (seen_macro1[0] + seen_macro2[0])/2, 'recall_seen_macro', (seen_macro1[1] + seen_macro2[1])/2, 'f1_seen_macro', (seen_macro1[2] + seen_macro2[2])/2)


seen_micro1 = precision_recall_fscore_support(new_labels_test_seen, pp1, average = 'micro')
seen_micro2 = precision_recall_fscore_support(new_labels_test_seen, pp2, average = 'micro')
print('precision_seen_micro', (seen_micro1[0] + seen_micro2[0])/2, 'recall_seen_micro', (seen_micro1[1] + seen_micro2[1])/2, 'f1_seen_micro', (seen_micro1[2] + seen_micro2[2])/2)

unseen_macro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'macro')
unseen_micro = precision_recall_fscore_support(new_labels_test_unseen, preds_unseen, average = 'micro')

print('precision_unseen_macro', unseen_macro[0], 'recall_unseen_macro', unseen_macro[1], 'f1_unseen_macro', unseen_macro[2])
print('precision_unseen_micro', unseen_micro[0], 'recall_unseen_micro', unseen_micro[1], 'f1_unseen_micro', unseen_micro[2])


precision_seen_macro 0.9528142769618739 recall_seen_macro 0.9338405808003536 f1_seen_macro 0.9399977738067852
precision_seen_micro 0.9501870112206732 recall_seen_micro 0.9501870112206732 f1_seen_micro 0.9501870112206732
precision_unseen_macro 0.42959423649411715 recall_unseen_macro 0.47357738584586373 f1_unseen_macro 0.36739309652072827
precision_unseen_micro 0.42310122583091114 recall_unseen_micro 0.42310122583091114 f1_unseen_micro 0.42310122583091114
